In [1]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: autogenes 1.0.4 has a non-standard dependency specifier matplotlib>=3.0.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of autogenes or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from datasets import load_dataset
from tqdm import tqdm


In [2]:
dataset = load_dataset("squad_v2")

In [3]:
dataset["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,id,title,context,question,answers
0,5a550e5e134fea001a0e1878,Communications_in_Somalia,"In the early 2000s, print media in Somalia reached a peak in activity. Around 50 newspapers were published in Mogadishu alone during this period, including Qaran, Mogadishu Times, Sana'a, Shabelle Press, Ayaamaha, Mandeeq, Sky Sport, Goal, The Nation, Dalka, Panorama, Aayaha Nolosha, Codka Xuriyada and Xidigta Maanta. In 2003, as new free electronic media outlets started to proliferate, advertisers increasingly began switching over from print ads to radio and online commercials in order to reach more customers. A number of the broadsheets in circulation subsequently closed down operations, as they were no longer able to cover printing costs in the face of the electronic revolution. In 2012, the political Xog Doon and Xog Ogaal and Horyaal Sports were reportedly the last remaining newspapers printed in the capital. According to Issa Farah, a former editor with the Dalka broadsheet, newspaper publishing in Somalia is likely to experience a resurgence if the National Somali Printing Press is re-opened and the sector is given adequate public support.",When did Issa Farah stop working for Dalka?,"{'text': [], 'answer_start': []}"
1,572656badd62a815002e81f4,Brigham_Young_University,"Brigham Young University's origin can be traced back to 1862 when a man named Warren Dusenberry started a Provo school in a prominent adobe building called Cluff Hall, which was located in the northeast corner of 200 East and 200 North. On October 16, 1875, Brigham Young, then president of the LDS Church, personally purchased the Lewis Building after previously hinting that a school would be built in Draper, Utah in 1867. Hence, October 16, 1875 is commonly held as BYU's founding date. Said Young about his vision: ""I hope to see an Academy established in Provo... at which the children of the Latter-day Saints can receive a good education unmixed with the pernicious atheistic influences that are found in so many of the higher schools of the country.""",Where was Brigham Young's school originally believed to be located?,"{'text': ['Draper, Utah'], 'answer_start': [404]}"
2,56de350b4396321400ee269b,Southern_Europe,"Southern Europe's most emblematic climate is that of the Mediterranean climate, which has become a typically known characteristic of the area. The Mediterranean climate covers much of Portugal, Spain, Southeast France, Italy, Croatia, Albania, Montenegro, Greece, the Western and Southern coastal regions of Turkey as well as the Mediterranean islands. Those areas of Mediterranean climate present similar vegetations and landscapes throughout, including dry hills, small plains, pine forests and olive trees.",Which parts of Turkey are characterized by a Mediterranean climate?,"{'text': ['Western and Southern coastal regions'], 'answer_start': [268]}"
3,5709e96a4103511400d5949e,United_States_dollar,"From 1792, when the Mint Act was passed, the dollar was defined as 371.25 grains (24.056 g) of silver. Many historians[who?] erroneously assume gold was standardized at a fixed rate in parity with silver; however, there is no evidence of Congress making this law. This has to do with Alexander Hamilton's suggestion to Congress of a fixed 15:1 ratio of silver to gold, respectively. The gold coins that were minted however, were not given any denomination whatsoever and traded for a market value relative to the Congressional standard of the silver dollar. 1834 saw a shift in the gold standard to 23.2 grains (1.50 g), followed by a slight adjustment to 23.22 grains (1.505 g) in 1837 (16:1 ratio).[citation needed]",Who suggested that the ratio of silver to gold should be fixed?,"{'text': ['Alexander Hamilton'], 'answer_start': [284]}"
4,57276249dd62a815002e9be1,Cotton,"Cotton linters are fine, silky fibers which adhere to the seeds of the cotton plant after ginning. These curly fibers typically are less than 1⁄8 inch (3.2 mm) long. The term also may apply to the longer text

In [7]:
def preprocess_data(examples):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    max_length = 512  # Adjust based on your model's capacity

    questions = [q.strip() for q in examples['question']]
    contexts = [c.strip() for c in examples['context']]
    answers = examples['answers']

    inputs = tokenizer(questions, contexts, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')
    start_positions = []
    end_positions = []

    for i, answer in enumerate(answers):
        start_position = answer['answer_start'][0] if answer['answer_start'] else 0
        end_position = start_position + len(answer['text'][0]) if answer['text'] else 0

        start_positions.append(start_position)
        end_positions.append(end_position)
      # Ensure start and end positions are within bounds
    start_positions = [min(sp, max_length - 1) for sp in start_positions]
    end_positions = [min(ep, max_length - 1) for ep in end_positions]
    
    inputs.update({'start_positions': start_positions, 'end_positions': end_positions})
    return inputs

In [8]:
import logging
logging.disable(logging.WARNING)

In [9]:
tokenized_dataset = dataset.map(preprocess_data, batched=True)

In [10]:
print(tokenized_dataset['train'][0])

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}, 'input_ids': [101, 2043, 2106, 20773, 2707, 3352, 2759, 1029, 102, 20773, 21025, 19358, 22815, 1011, 5708, 100

In [11]:
import torch
from torch import nn

class LSTMForQuestionAnsweringWithDropout(nn.Module):
    def __init__(self, hidden_size, vocab_size, embedding_dim, num_layers, dropout_rate):
        super(LSTMForQuestionAnsweringWithDropout, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(hidden_size * 2, 2)  # For start and end positions

    def forward(self, input_ids, attention_mask):
        embeddings = self.embedding(input_ids)
        lstm_output, _ = self.lstm(embeddings)
        lstm_output = self.dropout(lstm_output)
        logits = self.linear(lstm_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits


In [12]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = pad_sequence([torch.tensor(item['input_ids']) for item in batch], batch_first=True, padding_value=0)
    attention_mask = pad_sequence([torch.tensor(item['attention_mask']) for item in batch], batch_first=True, padding_value=0)
    start_positions = torch.tensor([item['start_positions'] for item in batch])
    end_positions = torch.tensor([item['end_positions'] for item in batch])

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'start_positions': start_positions, 'end_positions': end_positions}


In [13]:
# Hyperparameters
hidden_size = 256
vocab_size = 30522  # Based on BERT's vocab
embedding_dim = 128
num_layers = 2
learning_rate = 5e-4
batch_size = 16
num_epochs = 10

# Initialize the model, optimizer, and loss function
model = LSTMForQuestionAnsweringWithDropout(hidden_size, vocab_size, embedding_dim, num_layers, dropout_rate=0.3)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

train_loader = DataLoader(tokenized_dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)


# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

cuda


LSTMForQuestionAnsweringWithDropout(
  (embedding): Embedding(30522, 128)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (linear): Linear(in_features=512, out_features=2, bias=True)
)

In [14]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Optimizer
initial_learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=initial_learning_rate)

# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in train_loader:
        # Assuming batch is a dictionary with the following structure
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        optimizer.zero_grad()
        start_logits, end_logits = model(input_ids, attention_mask)

        start_loss = loss_fn(start_logits, start_positions)
        end_loss = loss_fn(end_logits, end_positions)
        loss = (start_loss + end_loss) / 2
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_samples += input_ids.size(0)

        # Calculate the number of correct predictions
        start_pred = torch.argmax(start_logits, dim=1)
        end_pred = torch.argmax(end_logits, dim=1)
        correct_predictions += ((start_pred == start_positions) & (end_pred == end_positions)).sum().item()

    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    scheduler.step(avg_loss)

    print(f'Epoch {epoch + 1}/{num_epochs} - Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

/home/sa.ekbote/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/10 - Avg Loss: 0.2571, Accuracy: 0.3450
Epoch 2/10 - Avg Loss: 0.2469, Accuracy: 0.3851
Epoch 3/10 - Avg Loss: 0.2416, Accuracy: 0.3961
Epoch 4/10 - Avg Loss: 0.2369, Accuracy: 0.4062
Epoch 5/10 - Avg Loss: 0.2312, Accuracy: 0.4167
Epoch 6/10 - Avg Loss: 0.2246, Accuracy: 0.4281
Epoch 7/10 - Avg Loss: 0.2176, Accuracy: 0.4366
Epoch 8/10 - Avg Loss: 0.2109, Accuracy: 0.4439
Epoch 9/10 - Avg Loss: 0.2048, Accuracy: 0.4498
Epoch 10/10 - Avg Loss: 0.1990, Accuracy: 0.4535


In [15]:
# Save the model at the end of training
model_save_path = 'models/lstmdropout_qa_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')

Model saved to models/lstmdropout_qa_model.pth


In [16]:
from torch.utils.data import DataLoader
# Assuming you have a validation dataset loaded in a variable named 'val_dataset'

# Define your DataLoader for the validation set
val_loader = DataLoader(tokenized_dataset['validation'], batch_size=16, shuffle=False,  collate_fn=collate_fn)

# Function to evaluate the model
def evaluate(model, val_loader, device):
    model.eval()  # Switch to evaluation mode
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():  # No gradients needed for evaluation, which saves memory and computations
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            start_logits, end_logits = model(input_ids, attention_mask)

            # Calculate loss
            start_loss = loss_fn(start_logits, start_positions)
            end_loss = loss_fn(end_logits, end_positions)
            loss = (start_loss + end_loss) / 2
            total_loss += loss.item()

            # Calculate the number of correct predictions
            start_pred = torch.argmax(start_logits, dim=1)
            end_pred = torch.argmax(end_logits, dim=1)
            correct_predictions += ((start_pred == start_positions) & (end_pred == end_positions)).sum().item()
            total_samples += input_ids.size(0)

    avg_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy

In [17]:
# Evaluate the model after training
val_loss, val_accuracy = evaluate(model, val_loader, device)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')


Validation Loss: 0.2764, Validation Accuracy: 0.3751
